In [4]:
import cv2 as cv

img = cv.imread('./data/model-citizen/test2.png')
#cv.imshow('image', img)


In [15]:
import os
import glob

gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
#cv.imshow('gray image', gray)

# Optional: improve contrast
gray = cv.equalizeHist(gray)

# Threshold (adaptive) – keep text white, background black
thresh = cv.adaptiveThreshold(
    gray, 255,
    cv.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv.THRESH_BINARY_INV,
    35, 10
)
cv.imshow('thresholded image', thresh)

# Morphology to clean noise (larger kernel)
kernel = cv.getStructuringElement(cv.MORPH_RECT, (3,3)) #bigger numbers add background blur
morph = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel, iterations=1)  # remove specks
morph = cv.morphologyEx(morph, cv.MORPH_CLOSE, kernel, iterations=1)  # connect strokes
cv.imshow('morphological image', morph)

# Use connected components instead of contours
num_labels, labels, stats, centroids = cv.connectedComponentsWithStats(morph, connectivity=8)

# Visualization
vis = cv.cvtColor(morph, cv.COLOR_GRAY2BGR)

os.makedirs("results", exist_ok=True)

for i in range(1, num_labels):  # skip background
    x, y, w, h, area = stats[i]

    # Filter blobs (remove tiny specks and very large areas)
    if area < 50 or w*h > 0.15*img.shape[0]*img.shape[1]:
        continue

    char_img = morph[y:y+h + 20, x:x+w]
    cv.imwrite(f'results/char_{i}.png', char_img)

    # Draw boxes for visualization
    cv.rectangle(vis, (x, y), (x+w, y+h + 10), (0, 255, 0), 1)

cv.imshow("characters", vis)

cv.waitKey(0)
cv.destroyAllWindows()

# Clean up: remove all files in the results folder after program ends
for f in glob.glob('results/*'):
    try:
        os.remove(f)
    except Exception as e:
        print(f"Could not remove {f}: {e}")

In [ ]:
# gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
# cv.imshow('gray image', gray)
# gray = cv.equalizeHist(gray)

# # Threshold (adaptive or Otsu)
# thresh = cv.adaptiveThreshold(gray, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, 
#                                cv.THRESH_BINARY_INV, 11, 2)

# # Morphological operations to clean noise
# kernel = cv.getStructuringElement(cv.MORPH_RECT, (2,2))
# cv.imshow('thresholded image', thresh)

# morph = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel)
# cv.imshow('morphological image', morph)

# # Invert the binary image so characters are white on black
# inverted = cv.bitwise_not(morph)

# # Find contours (now characters are white)
# contours, _ = cv.findContours(inverted, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
# cv.imshow('contoured image', contours)

# # Extract and save each character bounding box
# for i, cnt in enumerate(contours):
#     x, y, w, h = cv.boundingRect(cnt)
#     if w > 10 and h > 10:  # Filter small noise
#         char_img = inverted[y:y+h, x:x+w]
#         cv.imwrite(f'results/char_{i}.png', char_img)

# cv.waitKey(0)
# cv.destroyAllWindows()

error: OpenCV(4.12.0) :-1: error: (-5:Bad argument) in function 'imshow'
> Overload resolution failed:
>  - mat is not a numerical tuple
>  - Expected Ptr<cv::cuda::GpuMat> for argument 'mat'
>  - Expected Ptr<cv::UMat> for argument 'mat'
